In [68]:
from zipfile import ZipFile
from pathlib import Path

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

In [69]:
# import cupy 
# import cudf

In [70]:
df_dev = pd.read_csv(Path.cwd()/'data'/'DSL2122_january_dataset'/'development.csv')
df_eval = pd.read_csv(Path.cwd()/'data'/'DSL2122_january_dataset'/'evaluation.csv')

In [71]:
df_eval.shape

(74999, 5)

In [72]:
df = df_dev.append(df_eval)

In [73]:
# df['date'] = df['date'].\
#     astype('string')\
#     .str.split(' ')\
#     .apply(lambda x : ' '.join([x[i] for i in [1,2,3,5]]))\
#     .pipe(pd.to_datetime)

### Stopwords analysis: sklearn, nltk, stop_words

#### nltk

In [74]:
from nltk import download as nltk_download

nltk_download('stopwords')

from nltk.corpus import stopwords

nltk_stopwords = stopwords.words('english')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\edo_c\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


#### sklearn

In [75]:
from sklearn.feature_extraction import text

sklearn_stopwords = list(text.ENGLISH_STOP_WORDS)

## Preprocessing

User manual filter

In [76]:
df = df.loc[
    (~(df['user'] == 'lost_dog') &
    ~(df['user'] == 'webwoke') &
    ~(df['user'] == 'tweetpet') &
    ~(df['user'].str.contains('tweeteradder')) &
    ~(df['user'].str.contains('tweetfollow')) &
    ~(df['user'] == 'divxdownloads')) |
    df['sentiment'].isna()
]

In [77]:
df[df['sentiment'].isna()].shape

(74999, 6)

Tokenization

In [78]:
df['text_token'] = df['text'].str.split()

Removal of '@words', '&words', 'http:words'

In [79]:
df['text_noAt'] = df['text_token'].apply(lambda x : [i for i in x if not(i.startswith('@'))])\
    .apply(lambda x : [i for i in x if not(i.startswith('&'))])\
    .apply(lambda x : [i for i in x if not(i.startswith('http:'))])

Removal of '&amp', '&quot'

In [80]:
df['text_noAmpQuot'] = df['text_noAt']\
    .apply(lambda x : [i for i in x if '&amp' not in i])\
    .apply(lambda x : [i for i in x if '&quot' not in i])

Removal of repeated letters

In [81]:
a = ['ulaaa','bubu']
import re
[re.sub(r'(.)\1+', r'\1', word) for word in ['ulaaa','bubu']]

['ula', 'bubu']

In [82]:
import re 
df['text_duplLetters'] = df['text_noAmpQuot']\
    .apply(lambda x : [re.sub(r'(.)\1+', r'\1', word) for word in x])

Remove puntuation

In [83]:
import string as py_string
# df['text_noPunct'] = df['text_noAmpQuot']\
df['text_noPunct'] = df['text_duplLetters']\
    .apply(lambda x : [i.translate(str.maketrans('', '', py_string.punctuation)) for i in x])\
    .apply(lambda x : [i for i in x if i != ''])

Stemming and lemmatization

In [84]:
from nltk.stem import PorterStemmer, LancasterStemmer, SnowballStemmer, WordNetLemmatizer

# nltk_download('wordnet')
# nltk_download('omw-1.4')
# nltk_lemmatizer = WordNetLemmatizer()

nltk_stemmer = PorterStemmer()
# nltk_stemmer = LancasterStemmer()
# nltk_stemmer = SnowballStemmer('english') 

df['text_stemmed'] = df['text_noPunct'].apply(lambda x : [nltk_stemmer.stem(word) for word in x])
# df['text_stemmed'] = df['text_noPunct'].apply(lambda x : [nltk_lemmatizer.lemmatize(word) for word in x])

Negation

In [85]:
from nltk.sentiment.util import mark_negation

df['text_neg'] = df['text_noPunct'].apply(lambda x : mark_negation(x))
df['text_stemmed_neg'] = df['text_stemmed'].apply(lambda x : mark_negation(x))

Removal of stopwords

In [86]:
from stop_words import get_stop_words

stop_words_stopwords = get_stop_words('english')

considered_stopwords = stop_words_stopwords + [f'{word}_neg' for word in stop_words_stopwords]

df['text_noStopwords'] = df['text_neg'].apply(lambda x : [i for i in x if i not in considered_stopwords])

## Model creation

In [87]:
from sklearn.model_selection import ParameterGrid

linearSVC_params = {
    'penalty' : ['l1','l2'], #['l1','l2']
    'dual' : [False],
    'tol' : [1e-2,1e-3],
    'fit_intercept' : [False],
    'class_weight' : ['balanced'],
    'max_iter' : [50,100],
    'random_state' : [42],
    'C' : [1,10]
}

TfidfVectorizer_params = {
    'stop_words' : [None],
    'ngram_range' : [(1,3)],
    'max_features' : [None],
    'max_df' : [1.0,0.001],
    'min_df' : [1,0.000001],
    'binary' : [True,False],
    'norm' : ['l1','l2'],
    'use_idf' : [True],
    'smooth_idf' : [True,False],
    'sublinear_tf' : [False]
}


number_different_configurations = len(list(ParameterGrid(TfidfVectorizer_params)))*len(list(ParameterGrid(linearSVC_params)))

print(number_different_configurations, number_different_configurations*30/60/60)

512 4.266666666666667


# Grid search

In [88]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, classification_report, confusion_matrix

df_final = df
# df_final['text_final'] = df_final['text_noPunct'].apply(lambda x : ' '.join(x))
df_final['text_final'] = df_final['text_neg'].apply(lambda x : ' '.join(x))

mask_train_test = df_final['sentiment'].notna()

X_train_valid = df_final.loc[mask_train_test,:]['text_final'].values
y_train_valid = df_final.loc[mask_train_test,:]['sentiment'].values
X_test = df_final.loc[~mask_train_test,:]['text_final'].values

X_train, X_valid, y_train, y_valid = train_test_split(
    X_train_valid, 
    y_train_valid, 
    shuffle=True, 
    # stratify=y_train_valid, 
    train_size=0.9, 
    random_state=42
)

vectorizer = 'tfidf'
model = 'linearSVC'

pipe = Pipeline([
    (vectorizer, TfidfVectorizer()),
    (model, LinearSVC())
])

def params_for_GridSearchCV(params_IN,step_name):
    return {f'{step_name}__{key}':value for (key,value) in params_IN.items()}

params_all = params_for_GridSearchCV(TfidfVectorizer_params,vectorizer)
params_all.update(params_for_GridSearchCV(linearSVC_params,model))
print(params_all)

gscv = GridSearchCV(pipe, params_all, cv = 3, verbose = 4, scoring='f1_macro', n_jobs=6)

gscv.fit(X_train, y_train)

f1 = f1_score(y_valid, gscv.predict(X_valid),average='macro')
report = classification_report(y_valid, gscv.predict(X_valid))
confusion = confusion_matrix(y_valid, gscv.predict(X_valid))

print(f1)
print(report)
print(confusion)

from pathlib import Path
from datetime import datetime

results_name = Path.cwd()/'gscv_results'/'linearSVC_final_results.csv'

pd.DataFrame(gscv.cv_results_).to_csv(results_name)

{'tfidf__stop_words': [None], 'tfidf__ngram_range': [(1, 3)], 'tfidf__max_features': [None], 'tfidf__max_df': [1.0, 0.001], 'tfidf__min_df': [1, 1e-06], 'tfidf__binary': [True, False], 'tfidf__norm': ['l1', 'l2'], 'tfidf__use_idf': [True], 'tfidf__smooth_idf': [True, False], 'tfidf__sublinear_tf': [False], 'linearSVC__penalty': ['l1', 'l2'], 'linearSVC__dual': [False], 'linearSVC__tol': [0.01, 0.001], 'linearSVC__fit_intercept': [False], 'linearSVC__class_weight': ['balanced'], 'linearSVC__max_iter': [50, 100], 'linearSVC__random_state': [42], 'linearSVC__C': [1, 10]}
Fitting 3 folds for each of 512 candidates, totalling 1536 fits
0.8007485288333881
              precision    recall  f1-score   support

         0.0       0.77      0.77      0.77      9395
         1.0       0.83      0.83      0.83     12900

    accuracy                           0.81     22295
   macro avg       0.80      0.80      0.80     22295
weighted avg       0.81      0.81      0.81     22295

[[ 7255  2140]


In [90]:
results = pd.read_csv('gscv_results\\linearSVC_final_results2.csv')
print(results.loc[results['rank_test_score']==1]['params'].iloc[0])
print(results.loc[results['rank_test_score']==1]['params'].iloc[1])

{'linearSVC__C': 1, 'linearSVC__class_weight': 'balanced', 'linearSVC__dual': False, 'linearSVC__fit_intercept': False, 'linearSVC__max_iter': 50, 'linearSVC__penalty': 'l2', 'linearSVC__random_state': 42, 'linearSVC__tol': 0.001, 'tfidf__binary': True, 'tfidf__max_df': 1.0, 'tfidf__max_features': None, 'tfidf__min_df': 1, 'tfidf__ngram_range': (1, 3), 'tfidf__norm': 'l2', 'tfidf__smooth_idf': False, 'tfidf__stop_words': None, 'tfidf__sublinear_tf': False, 'tfidf__use_idf': True}
{'linearSVC__C': 1, 'linearSVC__class_weight': 'balanced', 'linearSVC__dual': False, 'linearSVC__fit_intercept': False, 'linearSVC__max_iter': 50, 'linearSVC__penalty': 'l2', 'linearSVC__random_state': 42, 'linearSVC__tol': 0.001, 'tfidf__binary': True, 'tfidf__max_df': 1.0, 'tfidf__max_features': None, 'tfidf__min_df': 1e-06, 'tfidf__ngram_range': (1, 3), 'tfidf__norm': 'l2', 'tfidf__smooth_idf': False, 'tfidf__stop_words': None, 'tfidf__sublinear_tf': False, 'tfidf__use_idf': True}


# Best models test

In [24]:
# from sklearn.model_selection import GridSearchCV
# from sklearn.pipeline import Pipeline
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.svm import LinearSVC
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import f1_score, classification_report, confusion_matrix

# df_final = df
# # df_final['text_final'] = df_final['text_noPunct'].apply(lambda x : ' '.join(x))
# df_final['text_final'] = df_final['text_neg'].apply(lambda x : ' '.join(x))

# mask_train_test = df_final['sentiment'].notna()

# X_train_valid = df_final.loc[mask_train_test,:]['text_final'].values
# y_train_valid = df_final.loc[mask_train_test,:]['sentiment'].values
# X_test = df_final.loc[~mask_train_test,:]['text_final'].values

# X_train, X_valid, y_train, y_valid = train_test_split(
#     X_train_valid, 
#     y_train_valid, 
#     shuffle=True, 
#     # stratify=y_train_valid, 
#     train_size=0.9, 
#     random_state=42
# )

# vectorizer = TfidfVectorizer(
#     binary = True, 
#     max_df = 1.0, 
#     min_df = 1, 
#     max_features = None, 
#     ngram_range = (1,1), 
#     norm = 'l1', 
#     smooth_idf = True, 
#     stop_words = None, 
#     sublinear_tf = False, 
#     use_idf = True
# )

# model = LinearSVC(
#     C = 1, 
#     class_weight = 'balanced', 
#     dual = False, 
#     fit_intercept = False, 
#     max_iter = 100, 
#     penalty = 'l1', 
#     random_state = 42, 
#     tol = 0.001
# )

# pipe = Pipeline([
#     ('tfidf', vectorizer),
#     ('linearSVC', model)
# ])

# pipe.fit(X_train, y_train)

# f1 = f1_score(y_valid, pipe.predict(X_valid),average='macro')
# report = classification_report(y_valid, pipe.predict(X_valid))
# confusion = confusion_matrix(y_valid, pipe.predict(X_valid))

# print(f1)
# print(report)
# print(confusion)

In [ ]:
model = LinearSVC(
    C = 1, 
    class_weight = 'balanced', 
    dual = False, 
    fit_intercept = False, 
    max_iter = 50, 
    penalty = 'l2', 
    random_state = 42, 
    tol = 0.001
)

vectorizer = TfidfVectorizer(
    binary = True, 
    max_df = 1.0, 
    max_features = None, 
    min_df = 1, 
    ngram_range = (1,3), 
    norm = 'l2', 
    smooth_idf = False, 
    stop_words = None, 
    sublinear_tf = False, 
    use_idf = True
)

# MODIFIED: norm and ngram_range

In [91]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, classification_report, confusion_matrix

df_final = df
# df_final['text_final'] = df_final['text_noPunct'].apply(lambda x : ' '.join(x))
# df_final['text_final'] = df_final['text_neg'].apply(lambda x : ' '.join(x))
df_final['text_final'] = df_final['text_stemmed_neg'].apply(lambda x : ' '.join(x))
# df_final['text_final'] = df_final['text_noStopwords'].apply(lambda x : ' '.join(x))

mask_train_test = df_final['sentiment'].notna()

X_train_valid = df_final.loc[mask_train_test,:]['text_final'].values
y_train_valid = df_final.loc[mask_train_test,:]['sentiment'].values
X_test = df_final.loc[~mask_train_test,:]['text_final'].values

X_train, X_valid, y_train, y_valid = train_test_split(
    X_train_valid, 
    y_train_valid, 
    shuffle=True, 
    # stratify=y_train_valid, 
    train_size=0.9, 
    random_state=42
)

model = LinearSVC(
    C = 1, 
    class_weight = 'balanced', 
    dual = False, 
    fit_intercept = False, 
    max_iter = 50, 
    penalty = 'l2', 
    random_state = 42, 
    tol = 0.001
)

vectorizer = TfidfVectorizer(
    binary = True, 
    max_df = 1.0, 
    max_features = None, 
    min_df = 1, 
    ngram_range = (1,3), 
    norm = 'l2', 
    smooth_idf = False, 
    stop_words = None, 
    sublinear_tf = False, 
    use_idf = True
)

pipe = Pipeline([
    ('tfidf', vectorizer),
    ('linearSVC', model)
])

pipe.fit(X_train, y_train)

f1 = f1_score(y_valid, pipe.predict(X_valid),average='macro')
report = classification_report(y_valid, pipe.predict(X_valid))
confusion = confusion_matrix(y_valid, pipe.predict(X_valid))

print(f1)
print(report)
print(confusion)

0.8019786529108555
              precision    recall  f1-score   support

         0.0       0.77      0.78      0.77      9395
         1.0       0.84      0.83      0.83     12900

    accuracy                           0.81     22295
   macro avg       0.80      0.80      0.80     22295
weighted avg       0.81      0.81      0.81     22295

[[ 7282  2113]
 [ 2198 10702]]


Best configuration

In [299]:
model = LinearSVC(
    C = 1, 
    class_weight = 'balanced', 
    dual = False, 
    fit_intercept = False, 
    max_iter = 50, 
    penalty = 'l2', 
    random_state = 42, 
    tol = 0.001
)

vectorizer = TfidfVectorizer(
    binary = True, 
    max_df = 1.0, 
    max_features = None, 
    min_df = 1, 
    ngram_range = (1,3), 
    norm = 'l2', 
    smooth_idf = False, 
    stop_words = None, 
    sublinear_tf = False, 
    use_idf = True
)

+ NO stemming: 0.7953890537516292
+ Snowball: 0.7994796670641988
+ Lancaster: 0.7934227929849921
+ Porter: 0.7982780155296589
+ Wordnet: 0.7942085265529459

Results generation

In [92]:
pipe.fit(X_train_valid, y_train_valid)
y_pred = pipe.predict(X_test)
pd.DataFrame(y_pred.astype(int),columns=['Predicted']).to_csv("output_gram13_80197.csv",index_label="Id", header=["Predicted"])